# Assignment 1: Backpropagation

In [1]:
import numpy as np

In [42]:
inputs = []

for i in range(8):
    input = np.array([0] * 8)
    input[i] = 1
    inputs.append(input)

inputs = np.array(inputs*100)


In [54]:
W1 = np.random.rand(3, 8)
W2 = np.random.rand(8, 3)

def sigmoid(x):
    return 1/(1 + np.exp(-x))


def forward_prop(input):
    hidden = sigmoid(np.dot(W1, input))
    output = sigmoid(np.dot(W2, hidden))
    return(output)
    
def back_prop(expected, output):
    one = np.array([1]*8).reshape(-1,1)
    delta3 = output * (one - output) * (output - expected)
    return(delta3)

    # delta3 = np.dot((output-expected),np.dot(output,(one - output)))
    # delta2 = # bias, even for input layer!, also learning rate



In [55]:
a3 = forward_prop(inputs[0].reshape(-1,1))
print(a3)

delta3 = back_prop(inputs[0].reshape(-1,1), a3)
delta3

[[0.68692319]
 [0.71849993]
 [0.72569141]
 [0.74214909]
 [0.67127446]
 [0.64025593]
 [0.79040394]
 [0.6555982 ]]


array([[-0.06733021],
       [ 0.1453222 ],
       [ 0.14445859],
       [ 0.14202048],
       [ 0.14812682],
       [ 0.14746904],
       [ 0.1309427 ],
       [ 0.14802699]])